# Cyberbullying model using LSTM

In [1]:
!pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

## Preprocessing the dataset

In [3]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stop_words.update(list(string.punctuation))

In [4]:
df = pd.read_csv("bully.csv")
df.head()

,index,oh_label,Text
0,0,1,"""You fuck your dad."""
1,1,0,"""i really don't understand your point.\xa0 It ..."
2,2,0,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,3,0,"""listen if you dont wanna get married to a man..."
4,4,0,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [5]:
messages = df['Text']
y = df['oh_label']

In [6]:
df['Text']

0                                    "You fuck your dad."
1       "i really don't understand your point.\xa0 It ...
2       "A\\xc2\\xa0majority of Canadians can and has ...
3       "listen if you dont wanna get married to a man...
4       "C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...
                              ...                        
8794    "Never really gave it much thought. I just fig...
8795                   "Nadie se salva de la regla 34 xd"
8796                 "Question: Are you a boy or a girl?"
8797    "Leave your email or phone number and maybe yo...
8798    "From the scenarios you present, I see you bel...
Name: Text, Length: 8799, dtype: object

In [7]:
def get_simple_pos(tag) :
    if tag.startswith('J') :
        return wordnet.ADJ
    elif tag.startswith('V') :
        return wordnet.VERB
    elif tag.startswith('N') :
        return wordnet.NOUN
    elif tag.startswith('R') :
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(review) :
    global max_len
    words = word_tokenize(review)
    output_words = []
    for word in words :
        if word.lower() not in stop_words :
            pos = pos_tag([word])
            clean_word = lemmatizer.lemmatize(word,pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    max_len = max(max_len, len(output_words))
    return " ".join(output_words)

In [8]:
max_len = 0

In [9]:
print(messages[0])
messages = [clean_text(message) for message in messages]
print(messages[0])

"You fuck your dad."
`` fuck dad ''


In [10]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as file:
        word_to_vec_map = {}
        word_to_index = {}
        index_to_word = {}
        index = 0
        for line in file:
            line = line.strip().split()
            curr_word = line[0]
            word_to_index[curr_word] = index
            index_to_word[index] = curr_word
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            index += 1
    return word_to_index, index_to_word, word_to_vec_map

In [11]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [12]:
def sentences_to_indices(X, word_to_index, max_len):
    m = len(X)
    X_indices = np.zeros((m, max_len))
    for i in range(m):
        sentence_words = [w.lower() for w in X[i].split()]
        j = 0
        for word in sentence_words:
            if word in word_to_index:
                X_indices[i, j] = word_to_index[word]
            j += 1
    return X_indices

## The LSTM model

In [13]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [14]:
def NLPModel(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    
    embeddings = pretrained_embedding_layer(word_to_vec_map, word_to_index)(sentence_indices)
    
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(1)(X)
    X = Activation('sigmoid')(X)
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

In [15]:
from keras.layers import Input, LSTM, Dropout
from keras.models import Model

def NLPModel(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(shape=input_shape, dtype='int32')
    embeddings = pretrained_embedding_layer(word_to_vec_map, word_to_index)(sentence_indices)

    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(len(classes), activation='softmax')(X)

    model = Model(inputs=sentence_indices, outputs=X)

    return model


In [1]:
from keras.layers import Input, LSTM, Dropout, Dense
from keras.models import Model

def NLPModel(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(shape=input_shape, dtype='int32')
    embeddings = pretrained_embedding_layer(word_to_vec_map, word_to_index)(sentence_indices)

    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(num_classes, activation='softmax')(X)  # Replace num_classes with the number of output classes

    model = Model(inputs=sentence_indices, outputs=X)

    return model


In [2]:
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [3]:
num_classes=2
model = NLPModel((max_len,), word_to_vec_map, word_to_index)

model.summary()

NameError: name 'max_len' is not defined

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(messages, y, random_state = 0, test_size = 0.1)

In [20]:
X_train_indices = sentences_to_indices(X_train, word_to_index, max_len)

In [ ]:
# Convert true labels to one-hot encoded format
from tensorflow.keras.utils import to_categorical
Y_train_encoded = to_categorical(Y_train)
# Compile the model with appropriate loss function and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_indices, Y_train_encoded, epochs=1, batch_size=32, shuffle=True)


 76/248 [========>.....................] - ETA: 14:44 - loss: 0.6365 - accuracy: 0.6772

In [ ]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len)

In [ ]:
loss, acc = model.evaluate(X_test_indices, Y_test)
print("Test accuracy = ", acc)

## Accuracy of LSTM: 67.25%

In [ ]:
model.save("model.h5")

## Predictions

In [ ]:
text = "suck it"
text = [clean_text(text)]
text

In [ ]:
text = sentences_to_indices(text, word_to_index, max_len)

In [ ]:
text

In [ ]:
model.predict(text)[0][0]

## Extras

In [ ]:
import pickle

In [ ]:
pickle.dump(word_to_index, open('word_to_index.pkl', 'wb'))